<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/notebooks/Squadrons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

!{sys.executable} -m pip install --force-reinstall --upgrade "git+https://github.com/HausReport/ClubRaiders.git#egg=craid"
import craid.eddb.loader.DataProducer as dp
import pandas as pd
import logging
import plotly.graph_objs as go
from numpy import *


logging.getLogger().addHandler(logging.StreamHandler())
logging.getLogger().level = logging.DEBUG

arrays = dp.getDataArrays()
df: pd.DataFrame = arrays['dataFrame']

  Cloning https://github.com/HausReport/ClubRaiders.git to /tmp/pip-install-d2qq_xz0/craid
  Running command git clone -q https://github.com/HausReport/ClubRaiders.git /tmp/pip-install-d2qq_xz0/craid
     |████████████████████████████████| 71kB 1.9MB/s 
     |████████████████████████████████| 3.4MB 10.1MB/s 
     |████████████████████████████████| 194kB 33.4MB/s 
     |████████████████████████████████| 1.8MB 32.3MB/s 
     |████████████████████████████████| 1.2MB 34.5MB/s 
     |████████████████████████████████| 102kB 7.9MB/s 
     |████████████████████████████████| 10.1MB 42.2MB/s 
     |████████████████████████████████| 71kB 4.5MB/s 
     |████████████████████████████████| 798kB 41.4MB/s 
     |████████████████████████████████| 14.6MB 286kB/s 
     |████████████████████████████████| 235kB 41.2MB/s 
     |████████████████████████████████| 184kB 44.5MB/s 
     |████████████████████████████████| 133kB 43.6MB/s 
     |████████████████████████████████| 450kB 35.7MB/s 
     |██████████████

Loading from AWS
1- downloading to: /tmp/smol-factions.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/smol-factions.jsonl.gz] to [/tmp/smol-factions.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443
https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /smol-factions.jsonl.gz HTTP/1.1" 200 15372
Found data file: /tmp/smol-factions.jsonl.gz
Read 659 lines of faction data
1- downloading to: /tmp/smol-systems_populated.jsonl.gz


0:197,001,216
0.5:198,717,440


2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/smol-systems_populated.jsonl.gz] to [/tmp/smol-systems_populated.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443
https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /smol-systems_populated.jsonl.gz HTTP/1.1" 200 25613
Found data file: /tmp/smol-systems_populated.jsonl.gz
Read 171 lines of systems data
Populated club system keys
Club system lookup set has {171} items
Populated faction instances: {1036} items.
1- downloading to: /tmp/smol-stations.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/smol-stations.jsonl.gz] to [/tmp/smol-stations.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443


1:200,859,648
2:202,129,408


https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /smol-stations.jsonl.gz HTTP/1.1" 200 133539
Found data file: /tmp/smol-stations.jsonl.gz
Read 768 lines of station data, adding 768

station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research B

3:202,661,888


Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
from craid.eddb.faction.SquadronXYZ import SquadronXYZ
from craid.club.regions.RegionFactory import RegionFactory
from django.contrib.admin.utils import flatten

In [3]:
sNames = []
sSystems = []
sRegions = []

for squad in SquadronXYZ.myDict.keys():
  systems = SquadronXYZ.myDict.get(squad)
  for sysName in systems.keys():
    sysLoc = systems.get(sysName)
    if sysLoc is None:
      continue
    sNames.append(squad)
    sSystems.append(sysName)
    sReg = RegionFactory.getRegion(sysLoc[0], sysLoc[1], sysLoc[2])
    sRegions.append(sReg.getNumber())

df = pd.DataFrame(list(zip(sNames, sSystems, sRegions)), 
               columns =['Squadron Name', 'System', 'Region']) 

In [4]:
df

,Squadron Name,System,Region
0,104th Phoenix Discovery Group,L 12-19,0
1,104th Phoenix Discovery Group,Woyo Mina,0
2,10th Fleet,Barnard's Star,4
3,10th Hawk Squadron,HIP 11728,2
4,10th Hawk Squadron,Mitnahas,2
...,...,...,...
16992,redback consortium,Wolf 1003,0
16993,space vans limited,60 Herculis,0
16994,space vans limited,Aguane,0
16995,space vans limited,Bany,0


In [5]:

def getSquadronsByRegion(region=1, cutoff=0) -> pd.DataFrame:
  tmp = df[ df['Region']==region].groupby(['Squadron Name']).agg( {'Region': 'count'}).reset_index()
  tmp.columns = ['Squadron Name','Systems in Region']
  tmp = tmp[ tmp['Systems in Region'] >= cutoff]
  return tmp.sort_values('Systems in Region', ascending=False)

In [6]:
getSquadronsByRegion(1,cutoff=2)

,Squadron Name,Systems in Region
6,Operation Ida,18
11,The Hive,17
0,Anti Xeno Initiative,11
7,Sirius Inc,11
5,Hanks Alliance of Anti-Xeno,2


In [7]:
foo = getSquadronsByRegion(1,2)
print("| Squadron | # Systems |")
print("| -------- | --------- |")

for index, row in foo.iterrows():
    print("|" + row['Squadron Name'] + "|" + str(row['Systems in Region']) + "|")

| Squadron | # Systems |
| -------- | --------- |
|Operation Ida|18|
|The Hive|17|
|Anti Xeno Initiative|11|
|Sirius Inc|11|
|Hanks Alliance of Anti-Xeno|2|


In [8]:
def getClubFactionsInRegion(num: int) -> str:
  dataFrame: pd.DataFrame = arrays['dataFrame']
  foo = dataFrame[ dataFrame['regionNumber']==num][['factionName']]
  foo = foo.drop_duplicates()
  bar = flatten(foo.values.tolist())
  return " - ".join(bar)

In [9]:
for region in RegionFactory.regionDict.values():
  regNum = region.getNumber()
  if regNum == 0:
    continue
  regName = region.getTitle()
  print(f"\n## Region # {regNum}: {regName}")
  clubFacs = getClubFactionsInRegion(regNum)
  print(f"_Regional Club Factions: {clubFacs}_\n")
  if regNum==3:
    cutoff = 0
  else:
    cutoff = 2
  foo = getSquadronsByRegion(regNum,cutoff)
  print("| Squadron | # Systems |")
  print("| -------- | --------- |")

  for index, row in foo.iterrows():
    print("|" + row['Squadron Name'] + "|" + str(row['Systems in Region']) + "|")


## Region # 1: The Merope Region
_Regional Club Factions: Sirius Mining Merope - Aegis Research - Janus Incorporated_

| Squadron | # Systems |
| -------- | --------- |
|Operation Ida|18|
|The Hive|17|
|Anti Xeno Initiative|11|
|Sirius Inc|11|
|Hanks Alliance of Anti-Xeno|2|

## Region # 2: The Rectangle Region
_Regional Club Factions: Worster Insurance - Sirius Hot2Cold - Sirius Hyperspace - Aegis Core_

| Squadron | # Systems |
| -------- | --------- |
|Pilot Syndicate 4|41|
|House of the Rising CERN|38|
|Simbad Regime|37|
|Close Encounters Corps|35|
|Nagii Union|33|
|Lootan Dominion|29|
|Sirius Inc|27|
|Errantry Alliance of Narvert|27|
|EXO|26|
|Cooper Industries|24|
|Cobra Wing|21|
|Hunters of the Void|20|
|HFR Corp|20|
|New Republic|19|
|Laniakea|19|
|Royal Phoenix Corporation|18|
|Ukrainian Pilots Federation|16|
|Milkyway Miners Union|16|
|SI Terraforming|15|
|USCE|15|
|Elite Outlaws|15|
|The Wild Bunch|13|
|11th Squadron|12|
|The Guardians of AeSir|12|
|160 S.O.A.R.|12|
|Inters

In [10]:
def getView(reg, dataFrame):
  return dataFrame[dataFrame.apply(lambda x: reg.contains(x.x, x.y, x.z), axis=1)]


In [11]:
#from craid.appHelpers import AnnoyingCrap

squad = "Sirius Inc"
reg = RegionFactory.getSquadronRegion(squad,  64, "rgb(0,0,255)")
frame : pd.DataFrame = arrays['dataFrame']
view = getView(reg, frame)
view
#view['htext'] = view[['systemName', 'factionName']].agg('\n'.join, axis=1)

foo= view[['factionName']]
foo = foo.drop_duplicates()
bar = flatten(foo.values.tolist())
#return 
" - ".join(bar)

'Sirius Hot2Cold - Sirius Hyperspace - Aegis Core - Sirius Mining Merope - Aegis Research - Janus Incorporated'

In [12]:
reg.getFigure(view)

In [13]:
view

,systemName,factionName,x,y,z,allegiance,isHomeSystem,population,influence,updated,control,vulnerable,sysId,facId,difficulty,missionScore,salesScore,explorationScore,mineralSalesScore,bountyHuntingScore,smugglingScore,piracyMurderScore,regionNumber
11,BD+33 801,Sirius Hot2Cold,-31.31250,-25.03125,-107.87500,Independent,False,35800,10.7283,2020-07-04,False,,1980,14917,0.2,100.0,20.0,20.0,0,0.0,0.0,0.0,2
17,Daik,Sirius Hyperspace,-69.06250,-22.37500,-148.09375,Independent,False,2200,65.6000,2020-06-10,True,,3975,7327,0.6,100.0,20.0,20.0,0,55.0,0.0,50.0,2
26,Hemaki,Sirius Hot2Cold,-38.87500,-22.56250,-115.50000,Independent,False,2362890899,6.2874,2020-07-06,False,,5865,14917,2259.6,200.0,40.0,40.0,0,0.0,0.0,0.0,2
78,Mooramba,Sirius Hyperspace,-68.62500,-17.50000,-149.12500,Independent,False,2752,44.8000,2020-07-07,True,,13823,7327,0.3,100.0,0.0,0.0,0,0.0,0.0,50.0,2
83,Ngalia,Sirius Hyperspace,-74.62500,-35.03125,-165.96875,Independent,True,9695033,9.7097,2020-06-30,False,CivilWar,14358,7327,999999.0,0.0,0.0,0.0,0,110.0,0.0,0.0,2
96,Ross 591,Sirius Hot2Cold,-32.18750,-28.09375,-111.37500,Independent,True,4100889,11.7588,2020-07-06,False,Election,16214,14917,999999.0,400.0,80.0,80.0,0,0.0,0.0,0.0,2
102,Socho,Aegis Core,-72.43750,-101.15625,-141.18750,Independent,True,2927099,6.8862,2020-07-06,False,Anarchy,17049,75779,999999.0,200.0,40.0,40.0,0,0.0,0.0,0.0,2
115,Atlas,Sirius Mining Merope,-76.71875,-147.34375,-344.43750,Independent,False,1200000,2.9970,2020-07-07,False,LowInf,19991,75776,0.0,600.0,160.0,160.0,3,0.0,0.0,0.0,1
116,Celaeno,Aegis Research,-81.09375,-148.31250,-337.09375,Independent,False,1200000,9.7902,2020-07-07,False,RecInfFail,20053,75760,4.3,600.0,160.0,160.0,3,50.0,0.0,0.0,1
117,Electra,Janus Incorporated,-86.06250,-159.93750,-361.65625,Independent,False,1200000,13.5354,2020-07-07,False,PendOutbreak,20550,75568,9.7,400.0,40.0,40.0,0,0.0,0.0,0.0,1
